In [ ]:
%pip install langchain
%pip install ctransformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install pypdf
%pip install python_dotenv
%pip install langchain-pinecone
%pip install langchain-community
%pip install langchain-core
%pip install langchain-huggingface

In [ ]:
from langchain.chains import RetrievalQA 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore as langchainpinecone
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from pinecone import Pinecone
import os

In [ ]:
PINECONE_API_KEY="Enter your pinecone KEY here"

In [ ]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [ ]:
extracted_data=load_pdf("data/")

In [ ]:
def text_spit(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_spit(extracted_data)
print("length of my chunk :",len(text_chunks))

In [ ]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings=download_hugging_face_embeddings()

In [ ]:
os.environ["PINECONE_API_KEY"] =PINECONE_API_KEY

In [ ]:
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="ENTER your index name here by copying or same as it is"
docsearch=langchainpinecone.from_texts([t.page_content for t in text_chunks],embedding=embeddings,index_name=index_name)

In [ ]:
prompt_templete="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_templete,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={"max_new_tokens" :512,
                          "temperature" : 0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt : ")
    result=qa.invoke({"query":user_input})
    print("Response : ",result["result"])